In [2]:
import numpy as np
from itertools import product
import pandas as pd
from numba import jit
import matplotlib.pyplot as plt
from particle_creation import *
from mass_asign import *
from grav_potential import *
from acceleration import *

In [48]:
ng = 16
num_par = 2

pars_pos = (np.random.rand(num_par, 2)*ng).astype(np.float32)
pars_vel = (np.random.rand(num_par, 2)).astype(np.float32)
# pars_mass = np.random.rand(num_par).astype(np.float32)*100
pars_mass = np.ones_like(pars_pos[:,0]).astype(np.float32)

# pars_pos = np.array([[8.1,11.1],[8.1,10.1]]).astype(np.float32)
# pars_vel = np.array([[10,0],[0,0]]).astype(np.float32)
# # pars_mass = np.ones_like(pars_pos[:,0]).astype(np.float32)
# pars_mass = np.array((0.00001,1)).astype(np.float32)

# pars_df = pd.DataFrame({'pos_y': pars_pos[:,0], 'pos_x': pars_pos[:,1],
#                         'vel_y': pars_vel[:,0], 'vel_x': pars_vel[:,1],
#                         'mass': pars_mass})
pars_df = pd.DataFrame({'pos_x': pars_pos[:,0], 'pos_y': pars_pos[:,1],
                        'vel_x': pars_vel[:,0], 'vel_y': pars_vel[:,1],
                        'mass': pars_mass, 'id': np.arange(num_par)})

# Info array: (num_par, 5)
# Position_x, Position_y, Velocity_x, Velocity_y, Mass
pars_info = pars_df.loc[:, ['pos_x', 'pos_y', 'vel_x', 'vel_y', 'mass']].values
par_list = par_create_2d(pars_df)

In [49]:
@jit(nopython=True) 
def cic_density_2d(pars_pos_mass, ng, h=1):
    dens = np.zeros((ng, ng)) # initialize density field
    pars_pos = pars_pos_mass[:,:2]
    dens = np.zeros((ng, ng)) # initialize density field
    pos_float = pars_pos/h - 0.5 # particle positions in float
    pos_floor = np.floor(pos_float).astype(np.int16) # particle positions in int
    pos_cel = pos_floor + 1 # ceiling of floating point index
    pos_star = pos_float - pos_floor # distance from floor
    for i in range(len(pars_pos)):
        for idx_shift in np.array([[0, 0], [0, 1], [1, 0], [1, 1]]): # Density change: loop over 4 cells
            rho_idx = pos_cel[i] - idx_shift
            rho_idx[rho_idx == ng] = 0            
            dens[rho_idx[0], rho_idx[1]] +=  [pos_star[i], 1-pos_star[i]][idx_shift[0]][0] * [pos_star[i], 1-pos_star[i]][idx_shift[1]][1] * pars_pos_mass[i,2]/ (h ** 2)
    return dens

In [50]:
pars_pos_mass = pars_info[:,[0,1,4]]
dens = cic_density_2d(pars_pos_mass, ng, h=1)

In [51]:
phi = grav_phi_2d(dens, h=1)

In [52]:
phi.shape

(16, 16)

In [54]:
acc_x, acc_y = cic_acc_2d(par_list,phi, h=1)

In [63]:
def acc_mesh(dens,h=1):
    ng = phi.shape[0]
    phi_pdc = np.pad(phi, pad_width=1,mode='wrap')
    acc_x_pdc, acc_y_pdc = np.gradient(phi_pdc, 1, edge_order=1)
    acc_x, acc_y= -1 * acc_x_pdc[1:-1, 1:-1], -1 * acc_y_pdc[1:-1, 1:-1]
    return acc_x, acc_y
@jit(nopython=True) 
def acc_par(pars_pos, acc_x, acc_y, h=1):
    par_acc_x, par_acc_y = np.zeros(len(pars_pos)), np.zeros(len(pars_pos))
    pos_float = pars_pos/h - 0.5 # particle positions in float
    pos_floor = np.floor(pos_float).astype(np.int16) # particle positions in int
    pos_cel = pos_floor + 1 # ceiling of floating point index
    pos_star = pos_float - pos_floor # distance from floor
    for i in range(len(pars_pos)):
        for idx_shift in np.array([[0, 0], [0, 1], [1, 0], [1, 1]]): # Density change: loop over 4 cells
            rho_idx = pos_cel[i] - idx_shift
            rho_idx[rho_idx == ng] = 0    
            par_acc_x[i] +=  [pos_star[i], 1-pos_star[i]][idx_shift[0]][0] * [pos_star[i], 1-pos_star[i]][idx_shift[1]][1] * acc_x[rho_idx[0], rho_idx[1]] * (h ** 2)
            par_acc_y[i] +=  [pos_star[i], 1-pos_star[i]][idx_shift[0]][0] * [pos_star[i], 1-pos_star[i]][idx_shift[1]][1] * acc_y[rho_idx[0], rho_idx[1]] * (h ** 2)
    return par_acc_x, par_acc_y

In [64]:
acc_x, acc_y = acc_mesh(phi, h=1)
par_acc_x, par_acc_y = acc_par(pars_info[:,[0,1]],acc_x, acc_y ,h=1)

In [69]:
par_acc_x,par_acc_y

(array([ 1.91058721e-16, -1.91058721e-16]),
 array([ 1.27167557e-15, -1.27167557e-15]))

In [71]:
np.array([par_acc_x, par_acc_y]).T

array([[ 1.91058721e-16,  1.27167557e-15],
       [-1.91058721e-16, -1.27167557e-15]])

In [67]:
par_list[0].acc

array([1.91058721e-16, 1.27167557e-15])

In [68]:
par_list[1].acc

array([-1.91058721e-16, -1.27167557e-15])